## Carga de datos

In [1]:
import os
import requests
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import sqlite3

In [2]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

## Lectura de datos

In [3]:
data = pd.read_csv("data/covertype/covertype_train.csv")
data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
1,2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
2,3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3,3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
4,2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

## Selección de Características

In [5]:
def feature_selection(df, k):
    
    # Split info
    X = data.drop(columns = "Cover_Type")
    y = data["Cover_Type"]
    
    # Get numeric columns
    X_num = X.select_dtypes(include = [int, float])
    
    # KBest Selector
    kbest = SelectKBest(f_classif, k=k)
    X_new = kbest.fit_transform(X_num,y)
    
    # Table
    table = pd.DataFrame(
        {
            "Columna" : [columna for columna in X_num.columns],
            "Retain" : [True if columna in list(kbest.get_feature_names_out()) else False for columna in X_num.columns]
        }
    )
    
    # Final dataframe
    final_df = X_num[kbest.get_feature_names_out()]
    
    return table, final_df

In [6]:
table, data_fs = feature_selection(data, k = 8)

In [7]:
table

,Columna,Retain
0,Elevation,True
1,Aspect,False
2,Slope,True
3,Horizontal_Distance_To_Hydrology,True
4,Vertical_Distance_To_Hydrology,True
5,Horizontal_Distance_To_Roadways,True
6,Hillshade_9am,True
7,Hillshade_Noon,True
8,Hillshade_3pm,False
9,Horizontal_Distance_To_Fire_Points,True


In [8]:
data_fs.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points
0,2991,7,67,11,1015,233,234,1570
1,2876,18,485,71,2495,192,202,1557
2,3171,2,277,9,4374,213,237,1052
3,3087,13,190,31,4774,193,221,752
4,2835,10,212,41,3596,231,242,3280


## Almacenamiento en base de datos

In [9]:
path = "data/sql/proyecto.db"
os.makedirs(os.path.dirname(path), exist_ok=True)
conexion = sqlite3.connect(path)
data_fs.to_sql('feature_selection', conexion, if_exists='replace', index=False)

Se verifica la conexión:

In [10]:
consulta = "SELECT * FROM feature_selection"
df = pd.read_sql_query(consulta, conexion)
df.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points
0,2991,7,67,11,1015,233,234,1570
1,2876,18,485,71,2495,192,202,1557
2,3171,2,277,9,4374,213,237,1052
3,3087,13,190,31,4774,193,221,752
4,2835,10,212,41,3596,231,242,3280


In [11]:
conexion.close()